## Interactive Widget to see future trajectories GAN generates

Widget is designed for Fetch Push and Fetch Pick And Place environments.

In [1]:
import torch
torch.cuda.set_device(0)

%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=4, linewidth=300, suppress=True)
from ipywidgets import interact
from ipywidgets.widgets import IntSlider
import joblib

from plot_utils import generate_trajectories, load_experiment
from envs.fetch_push import FetchPush
from envs.fetch_pick_and_place import FetchPickAndPlace

Define the experiment. Here we choose to load in the GANs trained on FetchPickAndPlace

In [2]:
#expt_name = "FetchPush"
expt_name = "FetchPickAndPlace"
#env = FetchPush(remove_gripper=True)
env = FetchPickAndPlace()
controller, planner = load_experiment(expt_name, param_name="final")
planning_args = planner.args
traj_len = 50

/home/henry/anaconda3/envs/deepRL/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.22 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


We use these along with the planner to gather a few trajectories.

In [3]:
num_trajs = 3

def gather_trajs(num_trajs, traj_len=50, render=False):
    trajectories = []
    for i in range(num_trajs):
        traj = {}
        obs = env.reset()
        if render:
            env.render()
        end_goal = obs["desired_goal"]
        traj["states"] = [obs["observation"]]
        traj["end_goal"] = end_goal
        traj["sim_state"] = [env.save_state()]
        traj["actions"] = []
        for j in range(traj_len-1):
            action, _ = planner.generate_next_action(obs["observation"],
                                                    end_goal,
                                                    controller.imagination,
                                                    env, **planning_args)
            traj["actions"].append(action)
            obs,_,_,_ = env.step(action)
            if render:
                env.render()
            traj["states"].append(obs["observation"])
            traj["sim_state"].append(env.save_state())
        print("Trajectory {} completed".format(i))
        trajectories.append(traj)
    return trajectories

trajectories = gather_trajs(num_trajs=num_trajs, render=False)

Trajectory 0 completed
Trajectory 1 completed
Trajectory 2 completed


In [4]:
def plot_trajectories(num_imagined_trajs, num_steps, traj_ind, timestep, plot_exact=False, end_points_only=False, 
                      object_only=False):
    generate_trajectories(controller.imagination, env, num_imagined_trajs, object=True, plot_exact=plot_exact,
                         end_points_only=end_points_only, object_only=object_only, 
                          start_state=trajectories[traj_ind]["states"][timestep],
                         start_env_state=trajectories[traj_ind]["sim_state"][timestep], fixed_axes=True,
                         end_goal=trajectories[traj_ind]["end_goal"], plot_model=False, num_steps=num_steps,)

### Visualising the Imagined Future Trajectories
The widget plots the position of the gripper (SOLID RED CIRCLE), the position of the object (WHITE CIRCLE WITH RED BORDER) and the target goal (SOLID BLACK CIRCLE). It then plots a randomly generated set of imagined future trajectories from the trained ensemble of GANs. The blue trajectories are the imagined trajectories of the gripper, and the red trajectories are the imagined object trajectories. If we select the "plot_exact" checkbox then the actual trajectories in the simulator (based on the imagined actions) are shown (black for the gripper trajectories, green for the object trajectories).

The first slider lets us choose the number of imagined trajectories to plot, and the second chooses how many future time steps the imagined trajectories cover. The traj_ind slider lets us choose amongst the number of trajectories that we have generated, and the timestep slider let's us move through the trajectory to choose the starting time step. Setting cts_upd to true means that the plot will update as you drag the slider, but may be slow and not perform well.

In [5]:
cts_upd = False

interact(plot_trajectories, num_imagined_trajs=IntSlider(min=1, max=100, value=10, step=1, title="Number of imagined trajs",continuous_update=cts_upd), 
         num_steps=IntSlider(min=1, max=traj_len, value=5, step=1, title="Imagined traj lengths", continuous_update=cts_upd), 
         traj_ind=IntSlider(min=0, max=len(trajectories)-1, value=0, step=1, title="Traj index",continuous_update=cts_upd),
         timestep=IntSlider(min=0, max=traj_len-1, value=0, step=1, title="Timestep", continuous_update=cts_upd), 
         plot_exact=False, object_only=False)

interactive(children=(IntSlider(value=10, continuous_update=False, description='num_imagined_trajs', min=1), I…

<function __main__.plot_trajectories(num_imagined_trajs, num_steps, traj_ind, timestep, plot_exact=False, end_points_only=False, object_only=False)>